In [ ]:
import numpy as np
import pandas as pd
import keywords as kwrd
import ast
from pprint import pprint
from natsort import index_natsorted, natsorted
import json, collections, re

In [ ]:
sdg_indicators = pd.read_json('https://unstats.un.org/SDGAPI/v1/sdg/Indicator/List')\
    .rename(columns={'goal': 'Goal', 'target':'Target', 
    'code':'Indicator','description':'Ind_desc'})
sdg_goals = pd.read_json('https://unstats.un.org/SDGAPI/v1/sdg/Goal/List?includechildren=false')\
    .rename(columns={'code': 'Goal', 'title':'Goal_title', 'description':'Goal_desc'})[['Goal','Goal_title','Goal_desc']]
sdg_targets = pd.read_json('https://unstats.un.org/SDGAPI/v1/sdg/Target/List?includechildren=false')\
    .rename(columns={'code': 'Target', 'goal':'Goal', 'description':'Target_desc'})[['Goal','Target','Target_desc']]

sdg_goals = sdg_goals.sort_values(by=['Goal'], key=lambda x: np.argsort(index_natsorted(sdg_goals['Goal'])))
sdg_targets = sdg_targets.sort_values(by=['Target'], key=lambda x: np.argsort(index_natsorted(sdg_targets['Target'])))
sdg_indicators = sdg_indicators.sort_values(by=['Indicator'], key=lambda x: np.argsort(index_natsorted(sdg_indicators['Indicator'])))


In [ ]:
sdg_goals.head(40)

In [ ]:
sdgmetadata_dict = collections.OrderedDict()
for goalrow in sdg_goals.itertuples():
    sdg = f'SDG {goalrow.Goal}'
    sdgmetadata_dict[sdg]={'title': goalrow.Goal_title, 'description': goalrow.Goal_desc}
    sdgmetadata_dict[sdg]['targets'] = { f'Target {targetrow.Target}' : 
        { 'description': targetrow.Target_desc,
            'indicators': { f'{indicatorrow.Indicator}' : {'description':indicatorrow.Ind_desc} 
            for indicatorrow in sdg_indicators[sdg_indicators.Target==targetrow.Target].itertuples()}
        }
        for targetrow in sdg_targets[sdg_targets.Goal==goalrow.Goal].itertuples()}

with open("sdgmetadata_dict.json", "w") as outfile:
    json.dump(sdgmetadata_dict, outfile)

In [ ]:
pprint(sdgmetadata_dict)

In [ ]:
indicators_map = kwrd.indicators
indicators_map['UN_Indicators'].head()

In [ ]:
indicators_map['UN_Indicators'] = indicators_map['UN_Indicators'].apply(lambda x: ast.literal_eval(x))

In [ ]:
test_df = pd.read_excel('output/GD61/output/6-results/mapping_.xlsx', index_col=0)[['Goal', 'Target']]
test_df.head()

In [ ]:
indicators_map = pd.merge(test_df, kwrd.indicators, how='left', on='Target').drop('Goal_y', axis=1).rename(columns={'Goal_x':'Goal'})
indicators_map.head(100)

In [ ]:
def makeindicatorsjson(resultstotal_df, metadata_dict='Default'):

    if metadata_dict=='Default':
        metadata_dict = sdgmetadata_dict
        
    
    indicators_dict = {"name": "sdgs", "goals": []}

    goals = natsorted(list(set(indicators_map['Goal'].values.tolist())))

    for goal in goals:
        goal_dict = {'name': goal, 'title':metadata_dict[goal]['title'], 'description':metadata_dict[goal]['description'],
        'targets' : []}
        indicators_dict['goals'].append(goal_dict)
    
        for row in resultstotal_df[resultstotal_df['Goal']==goal].itertuples():
                
            target_dict={}
            target = f'Target {row.Target}'
            target_dict['name'] = target
            mock_target = re.match(r'Target \d{1,2}.0', target)
            
            if mock_target:
                target_dict['description'] = metadata_dict[goal]['description']
            else:
                target_dict['description'] = metadata_dict[goal]['targets'][target]['description']
            
            target_dict['indicators'] = {'un':[]}

            for unind in row.UN_Indicators:
                if unind:
                    unind_dict={
                        'name':unind, 
                        'description':metadata_dict[goal]['targets'][target]['indicators']#[unind]['description']
                    }
                    target_dict['indicators']['un'].append(unind_dict)

            goal_dict['targets'].append(target_dict)      

    return indicators_dict

In [ ]:
test_indicators_dict = makeindicatorsjson(indicators_map)
print(test_indicators_dict)

In [ ]:
with open("green_deal_indicators.json", "w") as outfile:
    json.dump(test_indicators_dict, outfile)